This code is based on:

https://medium.com/@taposhdr/medical-image-analysis-with-deep-learning-i-23d518abf531

Revision Notes:

21APR2022: Replaced use of CV2 (OPENCV) with Pillow (PIL, Python Image Library); Uses Jupyter's built-in `display()` function for image display; else Image.show() will open new window


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors

from PIL import Image

In [ ]:
import ipywidgets as widgets
import pydicom as pdicom
import os
import glob
import copy
from ipywidgets import interact, interactive, fixed, interact_manual

%matplotlib inline

#### Helper printing functions

`plti` - plots the provided image.  
`pltm` - plots multiple images passed as numpy arrays.   
`sample_stack` - plots every x slice of a 3D image stack

In [ ]:
def plti(im, h=8, title='',cmap='gray',**kwargs):
    y = im.shape[0]
    x = im.shape[1]
    w = (y/x) * h
    plt.figure(figsize=(w,h))
    plt.imshow(im, interpolation="none",cmap=cmap, **kwargs)
    plt.axis('off')
    plt.title(title)
    
def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=3, size=12,cmap='gray'):
    fig,ax = plt.subplots(rows,cols,figsize=[size,size])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        if ind < stack.shape[2]:
            ax[int(i/cols),int(i % cols)].set_title('slice %d' % ind)
            ax[int(i/cols),int(i % cols)].imshow(stack[:,:,ind],cmap)
            ax[int(i/cols),int(i % cols)].axis('off')
    plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid

def pltm(images,h=8,titles=[],cmap='gray'):
    
    imNum = len(images)
    imX   = (imNum+1)//2
    
    y = images[1].shape[0]
    x = images[1].shape[1]
    w = (y/x) * h
    
    fig = plt.figure(figsize=(w,h))
    grid = ImageGrid(fig, 111,             # similar to subplot(111)
                     nrows_ncols=(imX, 2), # creates 2x2 grid of axes
                     axes_pad=0.25,           # pad between axes in inch.
                     )

    for i in range(imNum):
        grid[i].imshow(images[i],cmap=cmap)  # The AxesGrid object work as a list of axes.
        grid[i].axis('off')
        grid[i].set_xticks([])
        grid[i].set_yticks([])
        if len(titles) > i :
            grid[i].set_title(titles[i])    

This is a quick example showing how to import a graphic file and perform a KMeans transform on it. The KMeans example is for introduction of its use later in this notebook

Pillow is a Python library that replaces PIL. It can read and write a variety of graphics files and has routines for basic image transformations

https://pillow.readthedocs.io/en/stable/handbook/tutorial.html


In [ ]:
img = Image.open('Fast Puppy.JPG')
print(img.format, img.size, img.mode)
display(img)


Interactive widgets are described here:

https://ipywidgets.readthedocs.io/en/latest/examples/Using%20Interact.html

Note that later in this notebook, a different method is used, which produces cleaner code.

In [ ]:
def f(x1,x2):
    return x1,x2

layers = interactive(f, x1=(0,20),x2=(0,20))
print('Use this slider to select the number of bins \nto use in the image KMeans decimation')
display(layers)


The following code performs a KMeans clustering on the colors in the image passed in. The number of clusters is defined by `kmeans`. An example tutorial is here:

https://www.thepythoncode.com/article/kmeans-for-image-segmentation-opencv-python

OpenCV is not necessary. The image simply needs to be in a numpy array.


In [ ]:
from sklearn.cluster import KMeans

def compress(img,kmeans):
    h,w = img.size
    nImg = np.array(img)                                           #https://stackoverflow.com/questions/384759/how-to-convert-a-pil-image-into-a-numpy-array
    im_small_long = nImg.reshape((h * w, 3))

    print(f'h: {h}, w: {w}, numpy image: {nImg.shape}, reshaped image: {im_small_long.shape}')
    print(layers.result)

    km = KMeans(n_clusters=kmeans)

    km.fit(im_small_long)
    cc = km.cluster_centers_.astype(np.uint8)
    out = np.asarray([cc[i] for i in km.labels_]).reshape(nImg.shape)
    plti(out,title='After kMeans of '+str(kmeans)+' bins')

    return out

    

In [ ]:
out1 = compress(img,layers.result[0])
out2 = compress(img,layers.result[1])


In [ ]:
pltm([nImg,out1,out2],30,["Original","KMeans=" + str(layers.result[0]),"KMeans=" + str(layers.result[1])])

In [ ]:
#INPUT_FOLDER = '/Users/john/Jupyter/Image Processing/Matlab/Pat002'
#INPUT_FOLDER = '/Users/john/Programming/Work/AnnotatedScans/scansDICOM/'
#INPUT_FOLDER = '/Users/john/Programming/Work/Medtronic Samples/Pat002'
INPUT_FOLDER = '/Users/john/Programming/Work/LungCT-Diagnosis-R_075/R_075/'
patients = os.listdir(INPUT_FOLDER)
patients.sort()
#print(patients)

In [ ]:
lstFilesDCM = []

def load_scan2(path):
    for dirName, subdirList, fileList in os.walk(path):
        for filename in fileList:
            if ".dcm" in filename.lower():
                lstFilesDCM.append(os.path.join(dirName,filename))
#                 print(lstFilesDCM)

    lstFilesDCM.sort()
    return lstFilesDCM

first_patient = load_scan2(INPUT_FOLDER)
first_patient.sort()
#print(first_patient)


In [ ]:
RefDs = pdicom.read_file(lstFilesDCM[0])

#Load dimensions based on the number of rows, columns, and slices (along the Z axis)
ConstPixelDims = (int(RefDs.Rows), int(RefDs.Columns), len(lstFilesDCM)+1)

#Load spacing values (in mm)
ConstPixelSpacing = (float(RefDs.PixelSpacing[0]), float(RefDs.PixelSpacing[1]), float(RefDs.SliceThickness))

print('Dimensions ',ConstPixelDims)
print('Pixel Size ',ConstPixelSpacing)

In [ ]:
x = np.arange(0.0, (ConstPixelDims[0]+1)*ConstPixelSpacing[0], ConstPixelSpacing[0])
y = np.arange(0.0, (ConstPixelDims[1]+1)*ConstPixelSpacing[1], ConstPixelSpacing[1])
z = np.arange(0.0, (ConstPixelDims[2]+1)*ConstPixelSpacing[2], ConstPixelSpacing[2])

# print(x,y,z)

In [ ]:
# The array is sized based on 'ConstPixelDims'

ArrayDicom = np.zeros(ConstPixelDims, dtype=RefDs.pixel_array.dtype)

# loop through all the DICOM files
for filenameDCM in lstFilesDCM:
    
    # read the file
#    print(filenameDCM)
    ds = pdicom.read_file(filenameDCM)
    # Store the raw image data
    ArrayDicom[:, :, pdicom.read_file(filenameDCM).InstanceNumber] = ds.pixel_array
    

All the image processing work was started from :

http://www.degeneratestate.org/posts/2016/Oct/23/image-processing-with-numpy/

Use the <b>slider</b> below to set the slice number for the remainder of the processing...

In [ ]:
sliceNum = widgets.IntSlider()
sliceNum.max = len(lstFilesDCM)
display(sliceNum)

sliceNum.value = 87

In [ ]:
# plt.figure(dpi=1600)
# plt.axes().set_aspect('equal', 'datalim')
# plt.set_cmap(plt.gray())
# plt.pcolormesh(x, y, np.flipud(ArrayDicom[:, :, 87]))
# fig = plt.gcf()
# fig.set_size_inches(4,4)

# This code is for an interactive slider. It causes the picture to flash too much and the 
# function responds to every increment as the slider moves. Too slow, too annoying.

# def f(sliceNum):
#     plti(ArrayDicom[:, :, sliceNum],10)

# interact(f, sliceNum=(0,len(lstFilesDCM)))

plti(ArrayDicom[:, :, sliceNum.value],10,title = "Slice Number {}".format(sliceNum.value),cmap='gray')




In [ ]:
import math

def sigmoid(size,sigma):

    # Create square windows of size n by n from a Gaussian (h) and Gaussian derivative (hx, hy) of width sigma
    nn = int((size-1)/2)
    h = np.zeros((size,size))
    hx = np.zeros((size,size))
    hy = np.zeros((size,size))
    
    a = np.asarray([[x**2 + y**2 for x in range(-nn,nn+1)] for y in range(-nn,nn+1)])
    mx = np.asarray([[x for x in range(-nn,nn+1)] for y in range(-nn,nn+1)])
    my = np.asarray([[y for x in range(-nn,nn+1)] for y in range(-nn,nn+1)])
    
    h  = (( 1/(2*np.pi*sigma**2))* np.exp(-a/(2*sigma**2)));
    hy = ((my/(2*np.pi*sigma**4))* np.exp(-a/(2*sigma**2)))*159;
    hx = ((mx/(2*np.pi*sigma**4))* np.exp(-a/(2*sigma**2)))*159;

    h  = h/h.max()
    hx = hx/hx.max()
    hy = hy/hy.max()

    return h, hx, hy


In [ ]:
from scipy.signal import convolve2d

a, ax, ay = sigmoid(3,1.5)

slice = ArrayDicom[:, :, sliceNum.value]
im_conv = convolve2d(slice, a, mode="same", boundary="symm")
# plti(im_conv,title='Guassian Blur')

sx = convolve2d(slice, ax, mode="same", boundary="symm")
sy = convolve2d(slice, ay, mode="same", boundary="symm")
im_grad = (np.sqrt(sx*sx + sy*sy))
# plti(im_grad,title='Gradient')

histLine = 200
convLine = im_conv[histLine,:]
im_convHist = copy.deepcopy(im_conv)
im_convHist[histLine,:] = 1000
convInd = [i for i,v in enumerate(convLine) if v > 4000]

origLine = slice[histLine,:]
im_origHist = copy.deepcopy(slice)
im_origHist[histLine,:] = 100
origInd = [i for i,v in enumerate(origLine) if v > 4000]

gradLine = im_grad[histLine,:]
im_gradHist = copy.deepcopy(im_grad)
im_gradHist[histLine,:] = 3000
gradInd = [i for i,v in enumerate(gradLine) if v > 1000]
for i in range(len(gradInd)):
    if ((i>0) and (gradInd[i-1] != (gradInd[i]-1))) or i == 0 :
        im_gradHist[:,gradInd[i]] = 3000

print(len(convInd),len(gradInd),len(origInd))
print(gradInd)

f = plt.figure(figsize=(20,3))
ax = f.add_subplot(121)
ax2 = f.add_subplot(122)
ax.plot(origLine)
ax2.plot(gradLine)

# grab the pixel buffer and dump it into a numpy array
# X = np.array(fig.canvas.renderer.buffer_rgba())

pltm([im_origHist,im_gradHist],30,['Guassian Blur','Gradient'])

In [ ]:
from sklearn.cluster import KMeans

clusters = 10
# slice = ArrayDicom[:, :, sliceNum.value]
slice = im_conv
h,w = slice.shape[:2]

fz = slice

# Add the xy coordinates as features
fx = np.asarray([[x for x in range(0,w)] for y in range(0,h)])
fz = np.dstack((fz,fx))
# fx = np.roll(fx,-1, axis=1)
# fz = np.dstack((fz,fx))
# fx = np.roll(fx,2, axis=1)
# fz = np.dstack((fz,fx))
fy = np.asarray([[y for x in range(0,w)] for y in range(0,h)])
fz = np.dstack((fz,fy))
# fy = np.roll(fy,-1, axis=0)
# fz = np.dstack((fz,fy))
# fy = np.roll(fy,2, axis=0)
# fz = np.dstack((fz,fy))

# fi = np.asarray([[x*w + y for x in range(0,w)] for y in range(0,h)])
# fz = np.dstack((fz,fi))
# fy = np.roll(fi,-1, axis=0)
# fz = np.dstack((fz,fy))
# fy = np.roll(fi,1, axis=0)
# fz = np.dstack((fz,fy))
# fx = np.roll(fi,-1, axis=1)
# fz = np.dstack((fz,fx))
# fx = np.roll(fi,1, axis=1)
# fz = np.dstack((fz,fx))
fs = np.roll(slice,1,axis=0)
fz = np.dstack((fz,fs))
fs = np.roll(slice,-1,axis=0)
fz = np.dstack((fz,fs))
fs = np.roll(slice,1,axis=1)
fz = np.dstack((fz,fs))
fs = np.roll(slice,-1,axis=1)
fz = np.dstack((fz,fs))

# for z in range(0,fz.shape[2]):
#     print(fz[0:5,0:5,z])

features = fz.shape[2]

# String the image out into a line (h * w) x 1 - there's only one feature
im_small_long = fz.reshape((h * w, features))
im_small_wide = im_small_long.reshape((h,w,features))
km = KMeans(n_clusters=clusters)

km.fit(im_small_long)

cc = km.cluster_centers_.astype(np.uint8)
# The data index returned is feature number deep. Just keep 3 z dimensions to create 
# an RGB code
ccs = np.delete(cc,(list(range(3,features))), axis=1)

out = np.asarray([ccs[i] for i in km.labels_]).reshape((h,w,3))

plti(out)

In [ ]:
# This cell just recolorizes the k-means results to random colors. 
# It's just a way to try to get better colors quickly
rnd_cc = np.random.randint(0,256, size = (clusters,3))
out = np.asarray([rnd_cc[i] for i in km.labels_]).reshape((h,w,3))
plti(out)

The following code comes from:

https://www.kaggle.com/terravic/lung-segmentation


In [ ]:
slice = copy.deepcopy(ArrayDicom[:, :, sliceNum.value])
slice[slice == -2000] = 0
# print (lung.RescaleIntercept, lung.RescaleSlope, np.min(slice), np.max(slice))
print (np.min(slice), np.max(slice))

plt.imshow(slice, cmap=plt.cm.gray)

In [ ]:
from skimage.segmentation import clear_border
from skimage.measure import label,regionprops, perimeter
from skimage.morphology import disk, dilation, remove_small_objects, erosion, closing, reconstruction, binary_closing
from skimage.filters import roberts, sobel
from scipy import ndimage as ndi

def get_segmented_lungs(im):
    '''
    This funtion segments the lungs from the given 2D slice.
    '''
    # Convert into a binary image. 
    binary = im < 604
    
    # Remove the blobs connected to the border of the image
    cleared = clear_border(binary)

    # Label the image
    label_image = label(cleared)
    plti(cleared)
    plti(label_image)
    
    # Keep the labels with 2 largest areas
    areas = [r.area for r in regionprops(label_image)]
    areas.sort()
    print('All areas: ',areas)
    for region in regionprops(label_image):
        print ('Region area: {:6d} Largest area {}'.format(region.area, areas[-2]))
        if region.area < areas[-2]:
            for coordinates in region.coords:                
                label_image[coordinates[0], coordinates[1]] = 0
    binary = label_image > 0

    # Closure operation with disk of radius 12
    selem = disk(10)
    binary = binary_closing(binary, selem)
    
    # Fill in the small holes inside the lungs
    edges = roberts(binary)
    binary = ndi.binary_fill_holes(edges)

    # Superimpose the mask on the input image
    get_high_vals = binary == 0
    im[get_high_vals] = 0
    
    return im

slice = copy.deepcopy(ArrayDicom[:, :, sliceNum.value])
# slice = copy.deepcopy(ArrayDicom[:, :, sliceNum.value-1:sliceNum.value+1])
slice[slice == -2000] = 0
# print (lung.RescaleIntercept, lung.RescaleSlope, np.min(slice), np.max(slice))
print ('Slice min {}, max {}'.format(np.min(slice), np.max(slice)))

# plt.figure(figsize=(20, 10))
# plt.subplot(1, 2, 1)
# plt.imshow(slice, cmap=plt.cm.gray)
# plt.imshow(slice[:,:,1], cmap=plt.cm.gray)

segmented = get_segmented_lungs(slice)

# plt.subplot(1, 2, 2)
# plt.imshow(segmented, cmap=plt.cm.gray)


Labeling without removal of things touching the edges of the scan for a single slice

In [ ]:
slice = copy.deepcopy(ArrayDicom[:, :, sliceNum.value])

plt.hist(slice.flatten(), bins=50, color='c')
plt.xlabel("Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Convert to a binary mask after stripping off all the high Hounsfield values.
print('Hounsfield Units - Min: {:4d} Max: {:4d}'.format(np.min(slice),np.max(slice)))
slice[np.where(slice > 500)] = 0
slice[np.where(slice > 0)] = 1

plti(slice)

label_image = label(slice)
areas = [r.area for r in regionprops(label_image)]
centers = [r.centroid for r in regionprops(label_image)]
areas.sort()
n = 10
print('Total areas: ',len(regionprops(label_image)))
print('Largest {} areas: {}'.format(n,areas[-n:]))

for region in regionprops(label_image):
    if (region.area < areas[-4]) or (region.centroid[0] > 400) or (region.centroid[0] < 100):
        for coordinates in region.coords:                
            label_image[coordinates[0], coordinates[1]] = 0
        
# label_image = label(slice, neighbors=4, connectivity=2)
plti(label_image)


Same algorithm but for entire dicom stack

In [ ]:
slice = copy.deepcopy(ArrayDicom[:, :, :])
slice[np.where(slice > 3000)] = 0

plt.hist(slice.flatten(), bins=50, color='c')
plt.xlabel("Original Hounsfield Units (HU)")
plt.ylabel("Frequency")
plt.show()

# Convert to a binary mask after stripping off all the high Hounsfield values.
print('Hounsfield Units - Min: {:4d} Max: {:4d}'.format(np.min(slice),np.max(slice)))
slice[np.where(slice > 500)] = 0
slice[np.where(slice > 0)] = 1

plti(slice[:,:,87])

print('Labeling...')
label_image = label(slice)

# Limits on area and size to get rid of things too big or too small
minAreaPct = 0.05
totalArea = slice.shape[0] * slice.shape[1] * slice.shape[2]
areaLimit = totalArea * minAreaPct
maxSizePct = 0.7
xMax = label_image.shape[0]*maxSizePct
yMax = label_image.shape[1]*maxSizePct

print('Getting initial areas')
areas = [r.area for r in regionprops(label_image)]
areas.sort()
n = 10
print('Total areas: ',len(regionprops(label_image)))
print('Largest {} areas: {}'.format(n,areas[-n:]))

print('Removing small areas')
remainingRegions = []
for region in regionprops(label_image):
    xLen = region.bbox[3]-region.bbox[0]
    yLen = region.bbox[4]-region.bbox[1]
    if region.area < areaLimit or xLen > xMax or yLen > yMax :
        for coordinates in region.coords:
            label_image[coordinates[0], coordinates[1], coordinates[2]] = 0
    else:
        remainingRegions.append(region)

# for region in remainingRegions:
#     tmp_image = copy.deepcopy(label_image)
#     tmp_label = region.label
#     for otherR in remainingRegions:
#         if otherR.label != tmp_label:
#             for coordinates in otherR.coords:
#                 tmp_image[coordinates[0], coordinates[1], coordinates[2]] = 0
#     plti(tmp_image[:,:,87],title = "tmp_label {}, bbox {}, area {}".format(tmp_label,region.bbox,region.area),cmap=cmap)
            

print('Re-indexing remaining areas')
finalRegions = []
areaX = []
idx = 1
for region in remainingRegions:
    region.label = idx
    finalRegions.append(region)
    areaX.append(region.area)
    for coordinates in region.coords:                
        label_image[coordinates[0], coordinates[1], coordinates[2]] = idx
    idx += 1

areas = [r.area for r in finalRegions]
print('Total remaining regions: {}'.format(len(finalRegions)))
print('Largest {} areas: {}'.format(n,areas[-n:]))

cmap = colors.ListedColormap(['black','white', 'green', 'red', 'blue', 'orange'])

plti(label_image[:,:,87],title='label_image slice 1',cmap=cmap)

stepSize = 5
show_image = label_image[:,:,:]
sample_stack(show_image, rows=int(np.ceil(show_image.shape[2]/5/stepSize)), cols=5, size=18, start_with=0, show_every=stepSize,cmap=cmap)

# sample_stack(label_image, rows=6, cols=8, size=15)



In [ ]:
print(slice.shape[0] * slice.shape[1] * slice.shape[2])

In [ ]:
import pprint

fx = np.asarray([[x for x in range(0,w)] for y in range(0,h)])
fy = np.asarray([[y for x in range(0,w)] for y in range(0,h)])
fz = np.dstack((fx,fy))
fy = np.roll(fy,-1, axis=0)
fz = np.dstack((fz,fy))
fy = np.roll(fy,2, axis=0)
fz = np.dstack((fz,fy))

fz = np.dstack((slice,fz))
print(fz.shape)
print(fx[0:5,0:5])

fi = np.asarray([[x*w + y for x in range(0,w)] for y in range(0,h)])
fz = fi
pprint.pprint(fi.shape)
pprint.pprint(fz[0:5,0:5])
fy = np.roll(fi,-1, axis=0)
fz = np.dstack((fz,fy))
pprint.pprint(fz.shape)
pprint.pprint(fz[0:5,0:5,1])
fy = np.roll(fy,2, axis=0)
fz = np.dstack((fz,fy))
pprint.pprint(fz.shape)
print(fz[0:5,0:5,:])
print(fz[0:5,0:5,:])
print(list(range(3,features-1)))

In [ ]:
def f(x):
    return x

interact(f, x=(0,20))


In [ ]:
from matplotlib.backends.backend_agg import FigureCanvas
import matplotlib.pyplot as plt

# plt.plot([1, 2, 3])

canvas = plt.get_current_fig_manager().canvas

agg = canvas.switch_backends(FigureCanvas)
plt.plot([1, 2, 3])
agg.draw()
s, (width, height) = agg.print_to_buffer()

# Convert to a NumPy array.
X = np.frombuffer(s, np.uint8).reshape((height, width, 4))

# Pass off to PIL.
from PIL import Image
im = Image.frombytes("RGBA", (width, height), s)

# Uncomment this line to display the image using ImageMagick's `display` tool.
# im.show()


In [ ]:
pwd
